Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://underactuated.csail.mit.edu/intro.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- import packages used throughout the notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast (even on Colab) because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='560c2adace05eb20ebd78377582015d5b2d3859a', drake_version='0.25.0', drake_build='releases')

# Imports.
import matplotlib.pyplot as plt
import numpy as np
import pydrake.all

from pydrake.all import (AddMultibodyPlantSceneGraph, PlanarSceneGraphVisualizer)

import underactuated
from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend

In [ ]:
# For some mysterious reason, this needs to be run in a second cell for the change to persist.
plt_is_interactive = SetupMatplotlibBackend(['notebook']) # Use 'inline' to force non-interactive.

# Dynamics of the Double Pendulum

This first cell gives an example of how to run a simulation and animate the results.


In [ ]:
# Set up a block diagram with the robot (dynamics) and a visualization block.
builder = pydrake.systems.framework.DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

# Load the double pendulum from Universal Robot Description Format
parser = pydrake.multibody.parsing.Parser(plant, scene_graph)
parser.AddModelFromFile(underactuated.FindResource("models/double_pendulum.urdf"))
plant.Finalize()

builder.ExportInput(plant.get_actuation_input_port())
visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph,
                                                          xlim=[-2.8, 2.8],
                                                          ylim=[-2.8, 2.8],
                                                          show=plt_is_interactive))
builder.Connect(scene_graph.get_pose_bundle_output_port(),
                visualizer.get_input_port(0))

# The logger needs to be told to expect a 4-element input
# (the 4-element double pendulum state, in this case).
logger = builder.AddSystem(pydrake.systems.primitives.SignalLogger(4))
logger.DeclarePeriodicPublish(0.033333, 0.0)
builder.Connect(plant.get_state_output_port(), logger.get_input_port(0))

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = pydrake.systems.analysis.Simulator(diagram)

# Set the initial conditions
context = simulator.get_mutable_context()
context.SetContinuousState([1., 1., 0., 0.])  # (theta1, theta2, theta1dot, theta2dot)
diagram.get_input_port(0).FixValue(context, [0.,0.])   # Zero input torques

# Simulate and animate
AdvanceToAndVisualize(simulator, visualizer, 10.0)


In [ ]:
# Run this cell after running a simulation to plot traces
# of the pendulum state across time.
plt.figure()
fields = ["shoulder", "elbow"]
for i in range(2):
    plt.subplot(2, 1, i+1)
    plt.plot(logger.sample_times(), logger.data()[(i, i+2), :].transpose())
    plt.legend(["position", "velocity"])
    plt.xlabel('t')
    plt.ylabel(fields[i])
    plt.grid(True)
    plt.show()

It's worth taking a peek at the [file that describes the robot](https://github.com/RussTedrake/underactuated/blob/master/underactuated/models/double_pendulum.urdf). URDF and SDF are two of the standard formats, and they can be used to describe even very complicated robots (like the Boston Dynamics humanoid).

## Inspecting the dynamics (the manipulator equations)

We can also use Drake to evaluate the manipulator equations.  First we will evaluate the manipulator equations for a particular robot (with numerical values assigned for mass, link lengths, etc) and for a particular state of the robot.


In [ ]:

plant = pydrake.multibody.plant.MultibodyPlant(time_step=0)
parser = pydrake.multibody.parsing.Parser(plant)
parser.AddModelFromFile(underactuated.FindResource("models/double_pendulum.urdf"))
plant.Finalize()

# Evaluate the dynamics numerically
q = [0.1, 0.1]
v = [1, 1]
(M, Cv, tauG, B, tauExt) = underactuated.ManipulatorDynamics(plant, q, v)
print("M = \n" + str(M))
print("Cv = " + str(Cv))
print("tau_G = " + str(tauG))
print("B = " + str(B))
print("tau_ext = " + str(tauExt))

Drake is also fairly unique in supporting symbolic computation (with floating point coefficients).  Here is an example of printing out the symbolic dynamics of the double pendulum.  (If you've ever written out the equations of your robot, you know they get complicated quickly!)

In [ ]:
from pydrake.all import Variable

# Evaluate the dynamics symbolically
q = [Variable("theta0"), Variable("theta1")]
v = [Variable("thetadot0"), Variable("thetadot1")]
(M, Cv, tauG, B, tauExt) = underactuated.ManipulatorDynamics(plant.ToSymbolic(), q, v)
print("M = \n" + str(M))
print("Cv = " + str(Cv))
print("tau_G = " + str(tauG))
print("B = " + str(B))
print("tau_ext = " + str(tauExt))

We use a similar mechanisms to support automatic differentiation; we'll see examples of that soon!

# Feedback Cancellation of the Double Pendulum

Let's say that we would like our simple double pendulum to act like a
simple single pendulum (with damping), whose dynamics are given by:
\begin{align*} \ddot \theta_1 &= -\frac{g}{l}\sin\theta_1 -b\dot\theta_1 \\
\ddot\theta_2 &= 0. \end{align*} This is easily achieved using
(Note that our chosen dynamics do not actually stabilize $\theta_2$ -- this detail was left out for clarity, but would be necessary for any real
implementation.) $${\bf u}  = {\bf B}^{-1}\left[ {\bf C}\dot{{\bf q}} - {\bf \tau}_g +
{\bf M}\begin{bmatrix} -\frac{g}{l}s_1 - b\dot{q}_1 \\ 0 \end{bmatrix}
\right].$$ 

Since we are embedding a nonlinear dynamics (not a linear one), we refer
to this as "feedback cancellation", or "dynamic inversion".  This idea can,
and does, make control look easy - for the special case of a fully-actuated
deterministic system with known dynamics.  For example, it would have been
just as easy for me to invert gravity. Observe that the control derivations
here would not have been any more difficult if the robot had 100 joints.

## Acting like a single pendulum

First we implement our simple controller as a system that takes the pendulum state in, and outputs the motor torque.

In [ ]:
class Controller(pydrake.systems.framework.VectorSystem):
    """Defines a feedback controller for the double pendulum.

    The controller applies torques at the joints in order to:

    1) cancel out the dynamics of the double pendulum,
    2) make the first joint swing with the dynamics of a single pendulum, and
    3) drive the second joint towards zero.

    The magnitude of gravity for the imposed single pendulum dynamics is taken
    as a constructor argument.  So you can do fun things like pretending that
    gravity is zero, or even inverting gravity!
    """

    def __init__(self, multibody_plant, gravity):
        # 4 inputs (double pend state), 2 torque outputs.
        pydrake.systems.framework.VectorSystem.__init__(self, 4, 2)
        self.plant = multibody_plant
        self.g = gravity

    def DoCalcVectorOutput(self, context, double_pend_state, unused, torque):
        # Extract manipulator dynamics.
        q = double_pend_state[:2]
        v = double_pend_state[-2:]
        (M, Cv, tauG, B, tauExt) = underactuated.ManipulatorDynamics(self.plant, q, v)

        # Desired pendulum parameters.
        length = 2.
        b = .1

        # Control gains for stabilizing the second joint.
        kp = 1
        kd = .1

        # Cancel double pend dynamics and inject single pend dynamics.
        torque[:] = Cv - tauG - tauExt + M.dot(
            [self.g / length * np.sin(q[0]) - b * v[0], -kp * q[1] - kd * v[1]])


def simulate(gravity=-9.8):      
    builder = pydrake.systems.framework.DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

    # Load the double pendulum from Universal Robot Description Format
    parser = pydrake.multibody.parsing.Parser(plant, scene_graph)
    parser.AddModelFromFile(underactuated.FindResource("models/double_pendulum.urdf"))
    plant.Finalize()

    controller = builder.AddSystem(Controller(plant, gravity))
    builder.Connect(plant.get_state_output_port(), controller.get_input_port(0))
    builder.Connect(controller.get_output_port(0), plant.get_actuation_input_port())

    visualizer = builder.AddSystem(
        PlanarSceneGraphVisualizer(scene_graph, xlim=[-2.8, 2.8], ylim=[-2.8, 2.8], 
                                   show=plt_is_interactive))
    builder.Connect(scene_graph.get_pose_bundle_output_port(),
                    visualizer.get_input_port(0))

    diagram = builder.Build()

    # Set up a simulator to run this diagram
    simulator = pydrake.systems.analysis.Simulator(diagram)

    # Set the initial conditions
    context = simulator.get_mutable_context()
    context.SetContinuousState((1., 0., 0.2, 0.))  # (θ₁, θ₂, θ̇₁, θ̇₂)

    # Simulate
    AdvanceToAndVisualize(simulator, visualizer, 3.0)

If we simulate this system with the default parameters (gravity = -9.8m/s), then our double pendulum acts like a single pendulum.

In [ ]:
simulate()

But if we've gone this far, we could have replaced the dynamics with almost anything.  For instance, with a simple change, we can use feedback cancellation to invert gravity!

In [ ]:
simulate(gravity=9.8)